In [129]:
import pandas as pd
from utils import *
from joint_angles.calculate_joint_angles import *
#from joint_angles.utils import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
data_path = "../data/small_data/pose_saved.csv"
data = pd.read_csv(data_path)
print(data.dtypes)
data.head()

rgb_frameid     int64
kp3ds          object
kp2ds          object
bboxes         object
dtype: object


,rgb_frameid,kp3ds,kp2ds,bboxes
0,5000,"[[2433.527603034382, -1938.5729978558697, 65.7...","[[[1008.9975321765137, 101.44294880551615, 0.7...","[[743.9833984375, 0.0, 1234.8826904296875, 622..."
1,5001,"[[2446.607236443355, -1934.9092386957425, 67.5...","[[[999.6039120528574, 99.06533975351539, 0.758...","[[740.1781616210938, 0.0, 1231.525390625, 622...."
2,5002,"[[2460.826163175043, -1930.3790846162383, 69.3...","[[[992.6256386363831, 97.69814209614634, 0.727...","[[736.6728515625, 0.089996337890625, 1228.7199..."
3,5003,"[[2473.6984260388895, -1926.4497944254015, 73....","[[[985.1741546289431, 95.75803250340277, 0.752...","[[734.6681518554688, 0.10302734375, 1223.74499..."
4,5004,"[[2489.1893336653734, -1925.6113847420506, 73....","[[[978.2528523690814, 95.23409782663788, 0.765...","[[733.6686401367188, 0.173248291015625, 1216.8..."


### Let's only take the first two columns


In [92]:
df = data[['rgb_frameid', 'kp3ds']]
df.head()

,rgb_frameid,kp3ds
0,5000,"[[2433.527603034382, -1938.5729978558697, 65.7..."
1,5001,"[[2446.607236443355, -1934.9092386957425, 67.5..."
2,5002,"[[2460.826163175043, -1930.3790846162383, 69.3..."
3,5003,"[[2473.6984260388895, -1926.4497944254015, 73...."
4,5004,"[[2489.1893336653734, -1925.6113847420506, 73...."


In [93]:
print(type(df.kp3ds[0]))
df.kp3ds[0]

<class 'str'>


'[[2433.527603034382, -1938.5729978558697, 65.77853319876616, 0.715919840335846], [2455.1936794872177, -1931.8907779942238, 97.20523561103043, 0.6419889330863953], [2413.1285141606477, -1915.3594788777505, 100.405452670196, 0.6129101574420929], [2553.3050698424627, -1851.7950717623419, 114.90635783137866, 0.7434812635183334], [2406.47320246087, -1791.7937153377102, 130.30197906455479, 0.58548554033041], [2645.352619537392, -1677.0320842443841, -34.92217267652428, 0.721489417552948], [2305.953153691892, -1832.9165705828136, -22.7491032373318, 0.675788402557373], [2713.5425084514063, -1596.1945567660148, -283.31397363860276, 0.6877377976973852], [2205.742184616739, -2043.9587716844408, -78.7145570247363, 0.6339661717414856], [2746.769581107897, -1777.5664535076626, -405.71990057299956, 0.6417463049292564], [2196.6960493973734, -2288.6940364719085, -26.30439482351236, 0.7618403911590577], [2601.681535535075, -1730.8660615202673, -554.1895844242209, 0.642460686819894], [2415.360061812779, 

### Let's convert the string into vector

In [94]:
df['kp3ds'] = df['kp3ds'].apply(str2float)
df['kp3ds']

/var/folders/_q/5gxcvv9j1dg_pzh17qyfbkk40000gn/T/ipykernel_13915/1860301907.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['kp3ds'] = df['kp3ds'].apply(str2float)


0       [[2433.527603034382, -1938.5729978558697, 65.7...
1       [[2446.607236443355, -1934.9092386957425, 67.5...
2       [[2460.826163175043, -1930.3790846162383, 69.3...
3       [[2473.6984260388895, -1926.4497944254015, 73....
4       [[2489.1893336653734, -1925.6113847420506, 73....
                              ...                        
4995    [[2783.777098030839, -480.39212498854664, -45....
4996    [[2786.0874771535578, -479.94158441571295, -46...
4997    [[2791.0247204375196, -479.5124704274368, -46....
4998    [[2793.123929938673, -480.28451711442835, -46....
4999    [[2799.4689841766367, -479.83214699544794, -46...
Name: kp3ds, Length: 5000, dtype: object

### Let's separate each kp3ds line into columns for each body points, and remove kp3ds column

In [95]:
df_points = get_df_points(df)
df_points.head(2)

,point_0,point_1,point_2,point_3,point_4,point_5,point_6,point_7,point_8,point_9,point_10,point_11,point_12,point_13,point_14,point_15,point_16
0,"[2433.527603034382, -1938.5729978558697, 65.77...","[2455.1936794872177, -1931.8907779942238, 97.2...","[2413.1285141606477, -1915.3594788777505, 100....","[2553.3050698424627, -1851.7950717623419, 114....","[2406.47320246087, -1791.7937153377102, 130.30...","[2645.352619537392, -1677.0320842443841, -34.9...","[2305.953153691892, -1832.9165705828136, -22.7...","[2713.5425084514063, -1596.1945567660148, -283...","[2205.742184616739, -2043.9587716844408, -78.7...","[2746.769581107897, -1777.5664535076626, -405....","[2196.6960493973734, -2288.6940364719085, -26....","[2601.681535535075, -1730.8660615202673, -554....","[2415.360061812779, -1740.6648141592275, -579....","[2679.9952724415434, -1787.690377703567, -936....","[2400.8627281541926, -1778.5203763504858, -100...","[2756.2899011781155, -1799.6261441022816, -135...","[2414.4655146949895, -1684.1135292174695, -136..."
1,"[2446.607236443355, -1934.9092386957425, 67.53...","[2467.2217900211476, -1930.3347251724365, 97.9...","[2428.116065842915, -1907.5241451478714, 101.8...","[2569.8145999447324, -1852.6092353611177, 115....","[2422.253575661983, -1782.1572547434125, 129.7...","[2665.6696670464685, -1679.922232055294, -38.2...","[2318.2960842243237, -1820.4548819188512, -18....","[2725.426330672456, -1602.0102743236503, -296....","[2208.9420826541023, -2023.5744809148555, -71....","[2750.7631518119424, -1794.6567370184462, -407...","[2201.630793546887, -2268.631210247707, -14.05...","[2610.394828575268, -1748.6124215080272, -551....","[2419.367793916796, -1747.9863605084934, -570....","[2687.5016372386776, -1798.4410208952943, -933...","[2400.7339061752123, -1788.0900152347378, -100...","[2760.4205598737426, -1799.6442142374067, -135...","[2415.447631110754, -1698.3153128577876, -1366..."


In [96]:
df = pd.concat([df, df_points], axis=1)
df.drop(columns=['kp3ds'], inplace=True)
df.head(3)


,rgb_frameid,point_0,point_1,point_2,point_3,point_4,point_5,point_6,point_7,point_8,point_9,point_10,point_11,point_12,point_13,point_14,point_15,point_16
0,5000,"[2433.527603034382, -1938.5729978558697, 65.77...","[2455.1936794872177, -1931.8907779942238, 97.2...","[2413.1285141606477, -1915.3594788777505, 100....","[2553.3050698424627, -1851.7950717623419, 114....","[2406.47320246087, -1791.7937153377102, 130.30...","[2645.352619537392, -1677.0320842443841, -34.9...","[2305.953153691892, -1832.9165705828136, -22.7...","[2713.5425084514063, -1596.1945567660148, -283...","[2205.742184616739, -2043.9587716844408, -78.7...","[2746.769581107897, -1777.5664535076626, -405....","[2196.6960493973734, -2288.6940364719085, -26....","[2601.681535535075, -1730.8660615202673, -554....","[2415.360061812779, -1740.6648141592275, -579....","[2679.9952724415434, -1787.690377703567, -936....","[2400.8627281541926, -1778.5203763504858, -100...","[2756.2899011781155, -1799.6261441022816, -135...","[2414.4655146949895, -1684.1135292174695, -136..."
1,5001,"[2446.607236443355, -1934.9092386957425, 67.53...","[2467.2217900211476, -1930.3347251724365, 97.9...","[2428.116065842915, -1907.5241451478714, 101.8...","[2569.8145999447324, -1852.6092353611177, 115....","[2422.253575661983, -1782.1572547434125, 129.7...","[2665.6696670464685, -1679.922232055294, -38.2...","[2318.2960842243237, -1820.4548819188512, -18....","[2725.426330672456, -1602.0102743236503, -296....","[2208.9420826541023, -2023.5744809148555, -71....","[2750.7631518119424, -1794.6567370184462, -407...","[2201.630793546887, -2268.631210247707, -14.05...","[2610.394828575268, -1748.6124215080272, -551....","[2419.367793916796, -1747.9863605084934, -570....","[2687.5016372386776, -1798.4410208952943, -933...","[2400.7339061752123, -1788.0900152347378, -100...","[2760.4205598737426, -1799.6442142374067, -135...","[2415.447631110754, -1698.3153128577876, -1366..."
2,5002,"[2460.826163175043, -1930.3790846162383, 69.34...","[2484.918090524629, -1924.8040044837676, 99.79...","[2440.8411861266345, -1900.254950299179, 104.3...","[2584.0739331874606, -1851.960169894501, 113.9...","[2435.4652524619855, -1776.6510228370694, 132....","[2684.776549436161, -1687.3026095059545, -43.6...","[2329.3454519494003, -1808.3264969982044, -19....","[2743.056079751839, -1614.688905862936, -304.6...","[2195.1750290174386, -2006.137971398989, -65.8...","[2755.032244545425, -1812.6779647475068, -409....","[2206.279819898573, -2244.3941945026313, -3.28...","[2619.789612932464, -1751.2821608695785, -572....","[2427.258121931534, -1746.360628016013, -587.8...","[2695.004890749521, -1794.4201805342605, -942....","[2404.6687226515774, -1789.2744484614768, -100...","[2769.543079408299, -1809.2850525834021, -1367...","[2415.272491615325, -1700.7187023871097, -1364..."


#### The framerate is 30 FPS, let's convert the frameid column into time

In [97]:
df['rgb_frameid'] = (df['rgb_frameid'] - 5000)/30
df.rename(columns={'rgb_frameid': 'time'}, inplace=True)
df.head(2)

,time,point_0,point_1,point_2,point_3,point_4,point_5,point_6,point_7,point_8,point_9,point_10,point_11,point_12,point_13,point_14,point_15,point_16
0,0.000000,"[2433.527603034382, -1938.5729978558697, 65.77...","[2455.1936794872177, -1931.8907779942238, 97.2...","[2413.1285141606477, -1915.3594788777505, 100....","[2553.3050698424627, -1851.7950717623419, 114....","[2406.47320246087, -1791.7937153377102, 130.30...","[2645.352619537392, -1677.0320842443841, -34.9...","[2305.953153691892, -1832.9165705828136, -22.7...","[2713.5425084514063, -1596.1945567660148, -283...","[2205.742184616739, -2043.9587716844408, -78.7...","[2746.769581107897, -1777.5664535076626, -405....","[2196.6960493973734, -2288.6940364719085, -26....","[2601.681535535075, -1730.8660615202673, -554....","[2415.360061812779, -1740.6648141592275, -579....","[2679.9952724415434, -1787.690377703567, -936....","[2400.8627281541926, -1778.5203763504858, -100...","[2756.2899011781155, -1799.6261441022816, -135...","[2414.4655146949895, -1684.1135292174695, -136..."
1,0.033333,"[2446.607236443355, -1934.9092386957425, 67.53...","[2467.2217900211476, -1930.3347251724365, 97.9...","[2428.116065842915, -1907.5241451478714, 101.8...","[2569.8145999447324, -1852.6092353611177, 115....","[2422.253575661983, -1782.1572547434125, 129.7...","[2665.6696670464685, -1679.922232055294, -38.2...","[2318.2960842243237, -1820.4548819188512, -18....","[2725.426330672456, -1602.0102743236503, -296....","[2208.9420826541023, -2023.5744809148555, -71....","[2750.7631518119424, -1794.6567370184462, -407...","[2201.630793546887, -2268.631210247707, -14.05...","[2610.394828575268, -1748.6124215080272, -551....","[2419.367793916796, -1747.9863605084934, -570....","[2687.5016372386776, -1798.4410208952943, -933...","[2400.7339061752123, -1788.0900152347378, -100...","[2760.4205598737426, -1799.6442142374067, -135...","[2415.447631110754, -1698.3153128577876, -1366..."


### Placement of 3D body points
<a href=https://github.com/open-mmlab/mmpose/tree/main/projects/rtmpose><img src="https://user-images.githubusercontent.com/13503330/245036409-2417e4f7-2203-468f-bad0-e7a6a6bf8251.jpg"  width="150" height="300">

![alt text](img/3d_body_points_axes.jpeg "Title")

#### Which one is left/right ?
At the beginning of the video, the participants right foot has the lowest x value. By taking a look at points 15 and 16 x values, the **point 16** corresponds to the **right** foot.

From this information, we can better precise the column names in our dataset.

In [98]:
df.rename(columns = {'point_5':'shoulder_l', 'point_6':'shoulder_r', 
                     'point_8':'elbow_r', 'point_7':'elbow_l', 
                     'point_10':'wrist_r', 'point_9':'wrist_l',
                     'point_12':'hip_r', 'point_11':'hip_l'}, inplace = True)
df.head(2)

,time,point_0,point_1,point_2,point_3,point_4,shoulder_l,shoulder_r,elbow_l,elbow_r,wrist_l,wrist_r,hip_l,hip_r,point_13,point_14,point_15,point_16
0,0.000000,"[2433.527603034382, -1938.5729978558697, 65.77...","[2455.1936794872177, -1931.8907779942238, 97.2...","[2413.1285141606477, -1915.3594788777505, 100....","[2553.3050698424627, -1851.7950717623419, 114....","[2406.47320246087, -1791.7937153377102, 130.30...","[2645.352619537392, -1677.0320842443841, -34.9...","[2305.953153691892, -1832.9165705828136, -22.7...","[2713.5425084514063, -1596.1945567660148, -283...","[2205.742184616739, -2043.9587716844408, -78.7...","[2746.769581107897, -1777.5664535076626, -405....","[2196.6960493973734, -2288.6940364719085, -26....","[2601.681535535075, -1730.8660615202673, -554....","[2415.360061812779, -1740.6648141592275, -579....","[2679.9952724415434, -1787.690377703567, -936....","[2400.8627281541926, -1778.5203763504858, -100...","[2756.2899011781155, -1799.6261441022816, -135...","[2414.4655146949895, -1684.1135292174695, -136..."
1,0.033333,"[2446.607236443355, -1934.9092386957425, 67.53...","[2467.2217900211476, -1930.3347251724365, 97.9...","[2428.116065842915, -1907.5241451478714, 101.8...","[2569.8145999447324, -1852.6092353611177, 115....","[2422.253575661983, -1782.1572547434125, 129.7...","[2665.6696670464685, -1679.922232055294, -38.2...","[2318.2960842243237, -1820.4548819188512, -18....","[2725.426330672456, -1602.0102743236503, -296....","[2208.9420826541023, -2023.5744809148555, -71....","[2750.7631518119424, -1794.6567370184462, -407...","[2201.630793546887, -2268.631210247707, -14.05...","[2610.394828575268, -1748.6124215080272, -551....","[2419.367793916796, -1747.9863605084934, -570....","[2687.5016372386776, -1798.4410208952943, -933...","[2400.7339061752123, -1788.0900152347378, -100...","[2760.4205598737426, -1799.6442142374067, -135...","[2415.447631110754, -1698.3153128577876, -1366..."


## Intermediary features

### Get angles 

At first, I just tried to get the angles between 2 vectors (for example shoulder-elbow and wrist-elbow) to define it as the "angle joint". But then I realized that one can not just do that, since it will be then not sufficient to define the flexion, ab/adduction etc. 

I went back to read some literature on how to define joint angles based on 3D body points, and found some interesting references :
- [Method to estimate joint angles + code](https://temugeb.github.io/python/motion_capture/2021/09/16/joint_rotations.html)
It explains how to find joint angles based on basic body pose (T-pose)
- [This paper](https://www.nature.com/articles/s41591-022-02045-1#Sec9) explains that joint angles can be extracted in 3 d.o.f for each joint, by following the International Society of Biomechanics recommendations for Euler angle extractions of X (abduction/adduction), Y (internal/external rotation) and Z (flexion/extension).
- [This paper](https://www.rachelvitali.com/docs/Determining%20anatomical%20frames%20via%20inertial%20motion%20capture%20-%20a%20survey%20of%20methods.pdf) says that there are different methods used to define anatomical frames of reference for inertial motion capture.

--> So I should come up myself with a method to define the anatomical frame of reference.

**Here is my method to try to express the joint angles in every anatomical plane :**

1. Find a way to define body coordinates in order to easily select the anatomical planes (try reproduce the body coordinates as shown below in the picture)
2. Find a way to create the rotation matric that rotates the usual coordinate system to the body coordinates
3. Find a way to express the 3D point into body coordinates s.t. one can easily select the plane (for ex. Z-X plane for sagittal plane) to then compute the angle

<a href = "https://en.wikipedia.org/wiki/Anatomical_plane"><img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/2/24/Human_anatomy_planes%2C_labeled.svg/1200px-Human_anatomy_planes%2C_labeled.svg.png" width = 200/></a>

#### 1. Body coordinates system definition
Origin is at the middle of the hip joints

Z-axis : From origin to middle of shoulder joints

Y-axis : From origin to right hip joint

X-axis : Perpendicular to Z and Y, in head direction (point_0)
--> Take point_0, put same z coordinate as middle of hip

From [this method](https://temugeb.github.io/python/motion_capture/2021/09/16/joint_rotations.html), lets define the neutral body pose such as this : <img src = 'https://github.com/TemugeB/temugeb.github.io/blob/main/_posts/images/tpose.png?raw=true' widht = 10>

In [99]:
#We don't need the points around the head to calculate the joint angles
new_df = np.array(df.drop(columns=['time', 'point_0', 'point_1', 'point_2', 'point_3', 'point_4']))

#Code from elsewhere

#rotate to orient the pose better
R = get_R_z(np.pi/2)
# number of frames
for framenum in range(new_df.shape[0]):
    #number of points
    for pt_num in range(new_df.shape[1]):
        new_df[framenum, pt_num] = R @ new_df[framenum, pt_num]

kpts = convert_to_dictionary(new_df)
add_hips_and_neck(kpts)
filtered_kpts = median_filter(kpts)
get_bone_lengths(filtered_kpts)
get_base_skeleton(filtered_kpts)
calculate_joint_angles(filtered_kpts)

In [100]:
#We only need these joint angles at the end
keys = ['hips_angles', 'leftshoulder_angles', 'rightshoulder_angles', 'leftelbow_angles', 'rightelbow_angles']
df_angles = pd.DataFrame()
df_angles['time'] = df['time']

for key in keys:
    joint_angles = pd.DataFrame(filtered_kpts[key])
    #to remove the "_angles"
    df_angles[key[:-7]] = np.array(joint_angles.apply(np.array,1))*180/np.pi

df_angles.rename(columns={'hips' : 'trunk'}, inplace = True)
df_angles.head(3)

,time,trunk,leftshoulder,rightshoulder,leftelbow,rightelbow
0,0.000000,"[-156.3071409585956, 84.10757467315167, -103.2...","[-65.35052557971933, -6.067987861112847, 9.448...","[14.754037799458501, 5.47330924638175, 40.5281...","[-27.71587646697186, 16.76805966932435, -61.71...","[-24.047135031774356, -5.372450490608802, 24.8..."
1,0.033333,"[-175.28366682748134, 82.48075121466773, -91.5...","[-67.51159954100784, -7.384462023211777, 11.03...","[15.3400378492211, 5.288338736318286, 37.85600...","[-30.464382174507165, 18.896741612105956, -62....","[-22.416093504866875, -5.9151154925629355, 29...."
2,0.066667,"[176.50024574467085, 85.79969500008363, -89.16...","[-68.05919743430395, -7.1609873284539445, 10.5...","[14.310090315744985, 4.75483386463454, 36.6014...","[-29.957883718561263, 19.304924529493793, -64....","[-22.110321955397666, -7.363593270355689, 36.4..."


### Get angle velocities
Let's estimate the angular velocity of the right shoulder when the participant is picking the knife (flexion movement --> x axis)

In [118]:
w_shoulder_flex = angle_velocity(df_angles, 6, 7, name ='rightshoulder', axis = 0)
print(f'The participant does the shoulder flexion movement at {w_shoulder_flex:.2f} deg/s when picking a knife.')

The participant does the shoulder flexion movement at 41.07 deg/s when picking a knife.


Let's estimate the same angular velocity but when the participant does not move its shoulder

In [126]:
w1_shoulder_flex = angle_velocity(df_angles, 70, 71, name='rightshoulder', axis = 0)
print(f'The participant does the shoulder flexion movement at {w1_shoulder_flex:.2f} deg/s when doing nothing with the right shoulder')

The participant does the shoulder flexion movement at 2.15 deg/s when doing nothing with the right shoulder


### Trajectory distance

#### In what unit are the axises expressed to ? meters ? centimeters ?
Let's first try to look at the height of the person (point_1 to point_16)

In [125]:
a = df['point_1'][0]
b = (df['point_15'][0] + df['point_16'][0])/2
height = np.linalg.norm(a-b)
print(f'The person height (up to the nose) is {height:.2f}.') 

The person height (up to the nose) is 1472.17.


Looks like it is in millimeter, if we assume that the person measures ~1.47m from the feet to the nose level. 
Knowing this, the room experiment should have the following dimensions : 2m(width) x 5m(length) x 2m (height).

But this is not representative to the reality since the real dimensions of the labs are 2,37m(width) x 2,8m(length) ([from EPFL website](https://plan.epfl.ch/s/YQZs))

Maybe the resulting space grid does not really represent the real labs room dimension. But there is no way for now to come up with the real dimensions. Which is not really troublesome for now since we do not really look at the real distance measured at the end for our analysis.

NOTE : The algorithm that produces the 3D body points does not really seems okay, the resulting bodys does not turn to the right side. (ex. 0'00 and 2'34). Could be a problem if one wants to compute the trunk rotation

In [133]:
#lets look at the distance traveled by the right wrist when the person picks up the knife
distance = dist_trajectory(df, 6, 7, 'wrist_r')
print(f'The wrist traveled {distance:.2f} [cm or something] when the person picks up a knife')

The wrist traveled 692.76 [cm or something] when the person picks up a knife


### Distance A-B length and duration

In [120]:
def distance_AB(data, t1, t2, joint_name):
    idx = time_id(data, t1, t2)
    point_a = df[joint_name][idx[0]]
    point_b = df[joint_name][idx[1]]
    distance = np.linalg.norm(point_b - point_a)

    return distance

0       [2455.1936794872177, -1931.8907779942238, 97.2...
1       [2467.2217900211476, -1930.3347251724365, 97.9...
2       [2484.918090524629, -1924.8040044837676, 99.79...
3       [2494.3737814573888, -1922.4949403851683, 104....
4       [2509.9735892932163, -1919.9024906161483, 104....
                              ...                        
4995    [2768.2139351588753, -469.49892272444055, -32....
4996    [2768.271219478369, -470.2059453046999, -33.03...
4997    [2775.1101734844856, -470.6733268444091, -33.0...
4998    [2777.5056344211475, -470.6293645973954, -33.9...
4999    [2782.608139121164, -470.49442978892966, -32.8...
Name: point_1, Length: 5000, dtype: object

## Final features

### Get joint angles


#### 2.1 Shoulder flexion/extension
The angle is computed between the vector shoulder-elbow and shoulder-hip on sagittal plane.

<a href="https://physicaltherapyweb.com/hyper-extension-internal-rotation-heri-test/"><img src ="https://i0.wp.com/physicaltherapyweb.com/wp-content/uploads/2016/08/shoulder-flexion-and-extension.png?fit=329%2C410&ssl=1" width = 200/></a>

Let's choose a moment where the participant washes their hands, to target a moment where the shoulders only do flexion, with no abduction.



In [ ]:
idx = time_id(df, 25, 25.5)
filtered_kpts['leftshoulder_angles'][idx[0]:idx[1]]*180/np.pi

array([[-86.53340116,  24.48522178, -25.96304051],
       [-86.50942084,  24.39826947, -25.8816763 ],
       [-87.25647937,  24.69614683, -25.86808196],
       [-87.8280847 ,  24.67030801, -25.59283733],
       [-88.13367   ,  24.5721562 , -25.35997527],
       [-89.71005761,  24.14094294, -24.259799  ],
       [-88.92248848,  24.15001141, -24.59467671],
       [-88.24362344,  23.81319473, -24.53247535],
       [-83.85039639,  24.80637397, -27.52081329],
       [-80.13357357,  25.07515361, -29.62009702],
       [-75.99242849,  25.65771077, -32.50420793],
       [-72.79216457,  25.5477297 , -34.19082017],
       [-69.5095351 ,  25.76985606, -36.4935943 ],
       [-66.19549308,  25.90531932, -38.87043889],
       [-62.28601054,  26.07787743, -41.938733  ]])

Let's choose another moment where the participant is unpacking the tomatoes

In [ ]:
idx = time_id(df, 116, 116.5)
filtered_kpts['leftshoulder_angles'][idx[0]:idx[1]]*180/np.pi

array([[-80.22152227,  25.96487207, -30.61151294],
       [-79.98352415,  25.96405035, -30.73380028],
       [-77.81903949,  26.24106768, -32.21398228],
       [-76.26983154,  25.72840673, -32.43834914],
       [-76.2133695 ,  25.88611722, -32.66454537],
       [-75.95840168,  25.56875058, -32.41258784],
       [-75.1530636 ,  25.69163541, -33.0155592 ],
       [-74.27386417,  25.5012367 , -33.27293874],
       [-74.12153604,  25.70670878, -33.62225902],
       [-73.37949895,  25.95676087, -34.37532359],
       [-73.06746142,  25.94697912, -34.54709042],
       [-72.35689436,  25.95698481, -34.98447807],
       [-72.15394508,  25.6795708 , -34.7421728 ],
       [-71.80744579,  25.64142972, -34.89994038],
       [-71.55508634,  25.3296488 , -34.63831716]])

--> rotation around x-axis of human anatomical plan, as expected from extension movement

#### 2.2 Shoulder ab/adduction
In coronal plan

<a href='https://www.researchgate.net/publication/342490010_An_ergonomic_shoulder_for_robot-aided_rehabilitation_with_hybrid_control'><img src = 'https://www.researchgate.net/publication/342490010/figure/fig1/AS:960277239566349@1605959343300/Schematic-of-shoulder-abduction-adduction-and-vertical-flexion-extension.png' width = 300></a>

Let's check when the participant reach to the tomatoes in the fridge

In [ ]:
idx = time_id(df, 53, 53.5)
filtered_kpts['leftshoulder_angles'][idx[0]:idx[1]]*180/np.pi

array([[-46.39840671,  15.24554083, -34.6838049 ],
       [-46.05519195,  15.32862791, -35.13616408],
       [-46.17486705,  15.3586852 , -35.10598661],
       [-48.04124549,  14.77716622, -32.44706039],
       [-48.02149573,  14.63097006, -32.15348135],
       [-45.49133757,  15.1662791 , -35.23385792],
       [-45.17439485,  14.96174859, -35.0374174 ],
       [-46.61969892,  14.85500469, -33.66764347],
       [-46.11400738,  14.70544297, -33.73002309],
       [-45.11366884,  15.05827683, -35.30144089],
       [-44.62063159,  15.23148917, -36.09610067],
       [-44.24375614,  15.28538211, -36.53620608],
       [-43.41330718,  15.28865291, -37.26367532],
       [-43.2816072 ,  15.29960929, -37.40522559],
       [-42.9212793 ,  15.22143336, -37.54536359]])

It is harder to completely isolate the abduction movement since in real world, human always do a mix of flexion/abduction movement. At least we can see some rotation around z-axis that correspond to the abduction movement.

Let's check with another moment when the participant checks the tomatoes

In [ ]:
idx = time_id(df, 62, 62.5)
filtered_kpts['leftshoulder_angles'][idx[0]:idx[1]]*180/np.pi

array([[-44.17478204,  15.11364255, -36.2064142 ],
       [-44.37389317,  14.73994305, -35.19340682],
       [-44.91563263,  14.83869895, -34.97266285],
       [-46.43573623,  15.16000591, -34.46945236],
       [-47.30407252,  15.45578763, -34.43016027],
       [-47.59934916,  15.29475956, -33.8642796 ],
       [-48.65609517,  15.45809776, -33.41931711],
       [-49.38702313,  15.35239931, -32.67438737],
       [-50.41202542,  15.68153948, -32.6152143 ],
       [-51.16308624,  15.79391974, -32.31727083],
       [-51.19044177,  15.9826099 , -32.66867113],
       [-52.34789186,  16.94018055, -33.7123319 ],
       [-53.54293092,  16.99700879, -32.99608015],
       [-55.02096606,  17.14474685, -32.2855977 ],
       [-55.71511461,  17.21896879, -31.9705658 ]])

#### 3. Elbow flexion/extension

From litterature, the elbow joint allows us to perform flexion–extension and pronation–supination movement.

<a href="https://www.researchgate.net/figure/Normal-range-of-motion-for-the-elbow-joint_fig2_319499390"><img src="https://www.researchgate.net/publication/319499390/figure/fig2/AS:1086484513857552@1636049502528/Normal-range-of-motion-for-the-elbow-joint.jpg" alt="Normal range of motion for the elbow joint." width = "300" height = "300"/></a>

But since in our dataset, the pronation-supination movement is not captured, the joint angle of the elbow simply corresponds to the flexion-extension movement.
ref : Zwerus, E. L., Willigenburg, N. W., Scholtes, V. A., Somford, M. P., Eygendaal, D., & van den Bekerom, M. P. (2019). Normative values and affecting factors for the elbow range of motion. Shoulder & elbow, 11(3), 215–224. https://doi.org/10.1177/1758573217728711


Let's take the moment when the subjet reach the tomatoes in the fridge, left hand on the fridge door


In [ ]:
idx = time_id(df, 128.5, 129)
filtered_kpts['leftelbow_angles'][idx[0]:idx[1]]*180/np.pi

array([[  24.48645004,  -30.8296162 , -103.59381518],
       [  25.40523384,  -31.51363944, -102.75927635],
       [  26.64259205,  -32.26360874, -101.38931081],
       [  27.49377418,  -32.56465993, -100.10325554],
       [  27.17591337,  -32.55651314, -100.76914162],
       [  28.10682085,  -33.090874  ,  -99.76459969],
       [  30.54472385,  -34.07046366,  -96.58846218],
       [  30.46804237,  -33.77842051,  -96.21855683],
       [  30.72219959,  -33.68855367,  -95.56086421],
       [  29.84713781,  -33.13768044,  -96.2918723 ],
       [  30.32575499,  -33.77264342,  -96.48770987],
       [  30.40950263,  -33.74463299,  -96.27298443],
       [  30.96986027,  -34.05925009,  -95.74204702],
       [  31.43355843,  -34.25813639,  -95.20363396],
       [  31.78027189,  -34.66778221,  -95.26573854]])

Elbow flexion/extension around z-axis

#### 4.1 Trunk flexion

In [ ]:
idx = time_id(df, 93, 93.5)
filtered_kpts['hips_angles'][idx[0]:idx[1]]*180/np.pi

array([[ 31.08428412,  80.27502036, 109.22660216],
       [ 29.93613158,  80.49143327, 109.19488745],
       [ 29.91345037,  80.49801466, 109.08396428],
       [ 30.97826995,  80.56246258, 109.51800363],
       [ 34.02018848,  77.93764815, 106.04275606],
       [ 30.51243551,  80.60105737, 109.41299778],
       [ 31.32496049,  80.44158976, 109.39352152],
       [ 32.718285  ,  80.44332044, 109.83877712],
       [ 32.63809753,  80.37900645, 109.9843856 ],
       [ 33.0185936 ,  80.48376162, 109.90288483],
       [ 33.214289  ,  80.68146356, 110.80676818],
       [ 33.65450865,  80.1139788 , 109.69091465],
       [ 34.00882111,  80.05992878, 109.72129102],
       [ 34.50011262,  80.06992882, 109.57817449],
       [ 34.62578598,  80.22809217, 109.93952442]])

#### 4.2 Trunk rotation